#  Discrete Event Simulation: Implementation and Test

All we have left to do now is to implement the Simulation trait.
The idea is to keep in every instance of the Simulation trait an agenda of
actions to perform.
The agenda is a list of (simulated) events. Each event consists of an
action and the time when it must be produced.
The agenda list is sorted in such a way that the actions to be performed
first are in the beginning.

In [1]:
trait Simulation {
    type Action = () => Unit

    case class Event(time: Int, action: Action)

    private type Agenda = List[Event]

    private var agenda: Agenda = List()
    
    private var curtime = 0
    
    def currentTime: Int = curtime

    def afterDelay(delay: Int)(block: => Unit): Unit = {

        val item = Event(currentTime + delay, () => block)
        agenda = insert(agenda, item)
    }
    
    private def insert(ag: List[Event], item: Event): List[Event] = ag match {
        case first :: rest if first.time <= item.time =>
                first :: insert(rest, item)
        case _ =>
       item :: ag
    }
    private def loop(): Unit = agenda match {
        case first :: rest =>agenda = rest
        curtime = first.time
        first.action()
        loop()
        case Nil =>
    }
    
    def run(): Unit = {
        afterDelay(0) {
        println("*** simulation started, time = "+currentTime+" ***")
        }
        loop()
    }
}

defined trait Simulation

There is also a private variable, `curtime`, that contains the current simulation time:

`private var curtime = 0`
It can be accessed with a getter function `currentTime`:
```scala
def currentTime: Int = curtime
```
An application of the `afterDelay(delay)(block)` method inserts the task

`Event(curtime + delay, () => block)`

into the agenda list at the right position.

The event handling loop (method `loop`) removes successive elements from the agenda, and performs the associated actions.

The `run` method executes the event loop after installing an initial message that signals the start of simulation.

Before launching the simulation, we still need a way to examine the changes of the signals on the wires.

To this end, we define the function `probe`.

It’s convenient to pack all delay constants into their own trait which can
be mixed into a simulation. For instance:

In [2]:
trait Parameters {
def InverterDelay = 2
def AndGateDelay = 3
def OrGateDelay = 5
}

defined trait Parameters

In [3]:
abstract class Gates extends Simulation{
    def InverterDelay: Int
    def AndGateDelay: Int
    def OrGateDelay: Int
    
    class Wire {
        type Action = () => Unit
        private var sigVal = false
        private var actions: List[Action] = Nil
        def getSignal: Boolean = sigVal
        def setSignal(s: Boolean): Unit =
        if (s != sigVal) {
            sigVal = s
            actions foreach (_())
        }
        def addAction(a: Action): Unit = {
            actions = a :: actions
            a()
        }
    }
    
    def inverter(input: Wire, output: Wire): Unit = {
        def invertAction(): Unit = {
            val inputSig = input.getSignal
        afterDelay(InverterDelay) { output setSignal !inputSig }
        }
        input addAction invertAction
    }
    
    def andGate(in1: Wire, in2: Wire, output: Wire): Unit = {
        def andAction(): Unit = {
            val in1Sig = in1.getSignal
            val in2Sig = in2.getSignal
            afterDelay(AndGateDelay) { output setSignal (in1Sig & in2Sig) }
        }
        in1 addAction andAction
        in2 addAction andAction
    }
    
    def orGate(in1: Wire, in2: Wire, output: Wire): Unit = {
        def orAction(): Unit = {
            val in1Sig = in1.getSignal
            val in2Sig = in2.getSignal
        afterDelay(OrGateDelay) { output setSignal (in1Sig | in2Sig) }
        }
        in1 addAction orAction
        in2 addAction orAction
    }
    
    def probe(name: String, wire: Wire): Unit = {
        def probeAction(): Unit = {
            println(s"$name $currentTime value = ${wire.getSignal}")
        }
        wire addAction probeAction
    }
}

defined class Gates

In [4]:
abstract class Circuits extends Gates{
    
    def halfAdder(a: Wire, b: Wire, s: Wire, c: Wire): Unit = {
        val d = new Wire
        val e = new Wire
        orGate(a, b, d)
        andGate(a, b, c)
        inverter(c, e)
        andGate(d, e, s)
    }
    
    def fullAdder(a: Wire, b: Wire, cin: Wire, sum: Wire, cout: Wire): Unit = {
        val s = new Wire
        val c1 = new Wire
        val c2 = new Wire
        halfAdder(b, cin, s, c1)
        halfAdder(a, s, sum, c2)
        orGate(c1, c2, cout)
    }
}

defined class Circuits

Now let's run a concrete simulation

In [5]:
object sim extends Circuits with Parameters
import sim._

defined object sim
import sim._

In [6]:
val in1, in2, sum, carry = new Wire

in1: Wire = ammonite.$sess.cmd2$Helper$Gates$Wire@133d8e9
in2: Wire = ammonite.$sess.cmd2$Helper$Gates$Wire@84a36c
sum: Wire = ammonite.$sess.cmd2$Helper$Gates$Wire@7d60e8
carry: Wire = ammonite.$sess.cmd2$Helper$Gates$Wire@809a9c

In [7]:
halfAdder(in1, in2, sum, carry)
probe("sum", sum)
probe("carry", carry)

sum 0 value = false
carry 0 value = false


In [8]:
in1 setSignal true


In [9]:
run()

*** simulation started, time = 0 ***
sum 8 value = true


In [10]:
in2 setSignal true

In [11]:
run()

*** simulation started, time = 8 ***
carry 11 value = true
sum 16 value = false


In [12]:
in1 setSignal false
run()

*** simulation started, time = 16 ***
carry 19 value = false
sum 24 value = true


An alternative version of the OR-gate can be defined in terms of AND and INV
.


In [13]:
def orGateAlt(in1: Wire, in2: Wire, output: Wire): Unit = {
val notIn1, notIn2, notOut = new Wire
inverter(in1, notIn1); inverter(in2, notIn2)
andGate(notIn1, notIn2, notOut)
inverter(notOut, output)
}

defined function orGateAlt

**Question:**

What would change in the circuit simulation if the implementation of `orGateAlt` was used for OR?

State and assignments make our mental model of computation more complicated.

In particular, we lose referential transparency.

On the other hand, assignments allow us to formulate certain programs in an elegant way.

Example: discrete event simulation.

* Here, a system is represented by a mutable list of actions.
* The effect of actions, when they’re called, change the state of objects and can also install other actions to be executed in the future.